In [23]:
import numpy as np
import pandas as pd

app_dir = '/Users/nikhilvs/repos/nyu/covid-19'
COUNTRIES = [
    'United States', 
    'India', 
    'Italy', 
    'Canada', 
    'Spain', 
    'France', 
    'United Kingdom',
    'Iraq',
    'Japan',
    'South Korea',
    'Germany',
    'Brazil'
]

In [24]:
google_mobility_raw_df = pd.read_csv(os.path.join(app_dir, 'data/raw/google', 'Global_Mobility_Report.csv'))


In [25]:
google_mobility_raw_df['date'].isnull().values.any()

False

In [26]:
MOBILITY_COLUMNS_TO_KEEP = [
    'country_region', 
    'date', 
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline'
]

In [27]:
countries_mobility_df = google_mobility_raw_df[google_mobility_raw_df['country_region'].isin(COUNTRIES)][MOBILITY_COLUMNS_TO_KEEP]


In [28]:
countries_mobility_df.count()

country_region                                        259077
date                                                  259077
retail_and_recreation_percent_change_from_baseline    202056
grocery_and_pharmacy_percent_change_from_baseline     195007
parks_percent_change_from_baseline                     86859
transit_stations_percent_change_from_baseline         118406
workplaces_percent_change_from_baseline               249088
residential_percent_change_from_baseline              130902
dtype: int64

In [7]:
countries_mobility_df['retail_and_recreation_percent_change_from_baseline'].fillna(0.0, inplace = True)
countries_mobility_df['grocery_and_pharmacy_percent_change_from_baseline'].fillna(0.0, inplace = True)
countries_mobility_df['parks_percent_change_from_baseline'].fillna(0.0, inplace = True)
countries_mobility_df['transit_stations_percent_change_from_baseline'].fillna(0.0, inplace = True)
countries_mobility_df['workplaces_percent_change_from_baseline'].fillna(0.0, inplace = True)
countries_mobility_df['residential_percent_change_from_baseline'].fillna(0.0, inplace = True)

In [22]:
countries_mobility_df_agg = countries_mobility_df.groupby(['country_region', 'date']).mean()
countries_mobility_df_agg.to_csv(os.path.join(app_dir, 'data/staging', 'google_mobility.csv'))

In [45]:
oxford_stringency_df = pd.read_csv(os.path.join(app_dir, 'data/raw/oxford', 'OxCGRT_latest.csv'))

In [46]:
oxford_stringency_df.count()

CountryName                              22563
CountryCode                              22563
Date                                     22563
C1_School closing                        21783
C1_Flag                                   9701
C2_Workplace closing                     21543
C2_Flag                                   8167
C3_Cancel public events                  21453
C3_Flag                                   9758
C4_Restrictions on gatherings            21447
C4_Flag                                   8419
C5_Close public transport                21382
C5_Flag                                   6354
C6_Stay at home requirements             21295
C6_Flag                                   7458
C7_Restrictions on internal movement     21587
C7_Flag                                   7853
C8_International travel controls         21702
E1_Income support                        20828
E1_Flag                                   4112
E2_Debt/contract relief                  20806
E3_Fiscal mea

In [47]:
STRINGENCY_COLUMNS_TO_KEEP = [
    'CountryName',
    'Date',
    'C1_School closing',
    # 'C1_Flag',
    'C2_Workplace closing',
    # 'C2_Flag',
    'C3_Cancel public events',
    # 'C3_Flag',
    'C4_Restrictions on gatherings',
    # 'C4_Flag',
    'C5_Close public transport',
    # 'C5_Flag',
    'C6_Stay at home requirements',
    # 'C6_Flag',
    'C7_Restrictions on internal movement',
    # 'C7_Flag',
    'C8_International travel controls',
    'E1_Income support',
    # 'E1_Flag',
    'E2_Debt/contract relief',
    # 'E3_Fiscal measures',
    # 'E4_International support',
    'H1_Public information campaigns',
    # 'H1_Flag',
    'H2_Testing policy',
    'H3_Contact tracing',
    # 'H4_Emergency investment in healthcare',
    # 'H5_Investment in vaccines',
    'StringencyIndex'
]

In [48]:
countries_stringency_df = oxford_stringency_df[oxford_stringency_df['CountryName'].isin(COUNTRIES)][STRINGENCY_COLUMNS_TO_KEEP]
countries_stringency_df['Date'] = countries_stringency_df['Date'].transform(lambda d: str(d)[0:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
countries_stringency_df.fillna(0.0, inplace = True)
countries_stringency_df.to_csv(os.path.join(app_dir, 'data/staging', 'oxford_stringency.csv'))

In [13]:
countries_stringency_df.count()

CountryName                             1632
Date                                    1632
C1_School closing                       1632
C2_Workplace closing                    1632
C3_Cancel public events                 1632
C4_Restrictions on gatherings           1632
C5_Close public transport               1632
C6_Stay at home requirements            1632
C7_Restrictions on internal movement    1632
C8_International travel controls        1632
E1_Income support                       1632
E2_Debt/contract relief                 1632
H1_Public information campaigns         1632
H2_Testing policy                       1632
H3_Contact tracing                      1632
StringencyIndex                         1632
dtype: int64

In [59]:

jhu_cases_raw_df = pd.read_csv(os.path.join(app_dir, 'data/raw/jhu', 'time_series_covid19_confirmed_global.csv'))
countries_metrics_df = jhu_cases_raw_df.drop(columns = ['Province/State', 'Lat', 'Long'])
countries_metrics_df['Country/Region'] = countries_metrics_df['Country/Region'].replace('US', 'United States')

all_countries_metrics_cols = ['country', 'date', 'cases']
for shift in range(1, 20):
    all_countries_metrics_cols.append('cases_lag_' + str(shift))
all_countries_metrics_df = pd.DataFrame(columns = all_countries_metrics_cols)


for country in COUNTRIES:
    transposed_df = countries_metrics_df[countries_metrics_df['Country/Region'] == country].drop(columns = ['Country/Region']).transpose()
    num_cols = len(transposed_df.columns)
    num_rows = transposed_df.shape[0]

    if num_rows == 0 or num_cols == 0:
        continue

    value_col_name = transposed_df.columns[0]

    country_series = pd.Series([country] * num_rows)
    dates_series = pd.to_datetime(pd.Series(transposed_df.index))
    value_series = transposed_df[value_col_name].reset_index()[value_col_name]

    country_df = pd.DataFrame({
        'country': country_series,
        'date': dates_series,
        'cases': value_series
    })

    for shift in range(1, 20):
        country_df['cases_lag_' + str(shift)] = value_series.shift(shift, fill_value = 0)

    all_countries_metrics_df = all_countries_metrics_df.append(country_df, ignore_index = True)

all_countries_metrics_df.to_csv(os.path.join(app_dir, 'data/staging', 'jhu_cases.csv'), index = False)

In [63]:
value_col_name

217

In [56]:
all_countries_metrics_df[all_countries_metrics_df['country'] == 'United Kingdom']

,country,date,cases,cases_lag_1,cases_lag_2,cases_lag_3,cases_lag_4,cases_lag_5,cases_lag_6,cases_lag_7,...,cases_lag_10,cases_lag_11,cases_lag_12,cases_lag_13,cases_lag_14,cases_lag_15,cases_lag_16,cases_lag_17,cases_lag_18,cases_lag_19
684,United Kingdom,2020-01-22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
685,United Kingdom,2020-01-23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
686,United Kingdom,2020-01-24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
687,United Kingdom,2020-01-25,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
688,United Kingdom,2020-01-26,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,United Kingdom,2020-05-10,118,118,118,118,118,115,115,115,...,114,111,110,110,109,109,99,99,99,86
794,United Kingdom,2020-05-11,119,118,118,118,118,118,115,115,...,114,114,111,110,110,109,109,99,99,99
795,United Kingdom,2020-05-12,121,119,118,118,118,118,118,115,...,114,114,114,111,110,110,109,109,99,99
796,United Kingdom,2020-05-13,121,121,119,118,118,118,118,118,...,115,114,114,114,111,110,110,109,109,99


In [58]:
jhu_cases_raw_df[jhu_cases_raw_df['Country/Region'] == 'United Kingdom']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20
217,Bermuda,United Kingdom,32.3078,-64.7505,0,0,0,0,0,0,...,115,118,118,118,118,118,119,121,121,122
218,Cayman Islands,United Kingdom,19.3133,-81.2546,0,0,0,0,0,0,...,78,78,80,81,81,81,84,85,86,93
219,Channel Islands,United Kingdom,49.3723,-2.3644,0,0,0,0,0,0,...,545,545,545,545,545,545,546,547,548,549
220,Gibraltar,United Kingdom,36.1408,-5.3536,0,0,0,0,0,0,...,144,144,144,146,146,146,147,148,147,147
221,Isle of Man,United Kingdom,54.2361,-4.5481,0,0,0,0,0,0,...,326,327,329,329,329,330,330,331,332,332
222,Montserrat,United Kingdom,16.7425,-62.1874,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
223,NaN,United Kingdom,55.3781,-3.4360,0,0,0,0,0,0,...,194990,201101,206715,211364,215260,219183,223060,226463,229705,233151
248,Anguilla,United Kingdom,18.2206,-63.0686,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
249,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,6,7,7,7,7,7,7,7,7,7
250,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,...,12,12,12,12,12,12,12,12,12,12


In [64]:
transposed_df

,217,218,219,220,221,222,223,248,249,250,257
1/22/20,0,0,0,0,0,0,0,0,0,0,0
1/23/20,0,0,0,0,0,0,0,0,0,0,0
1/24/20,0,0,0,0,0,0,0,0,0,0,0
1/25/20,0,0,0,0,0,0,0,0,0,0,0
1/26/20,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5/10/20,118,81,545,146,330,11,219183,3,7,12,13
5/11/20,119,84,546,147,330,11,223060,3,7,12,13
5/12/20,121,85,547,148,331,11,226463,3,7,12,13
5/13/20,121,86,548,147,332,11,229705,3,7,12,13


In [37]:
value_series.shift(1, fill_value = 0)

0           0
1           0
2           0
3           0
4           0
        ...  
109    156061
110    162699
111    169594
112    178214
113    190137
Name: 28, Length: 114, dtype: int64

In [17]:

jhu_deaths_raw_df = pd.read_csv(os.path.join(app_dir, 'data/raw/jhu', 'time_series_covid19_deaths_global.csv'))
countries_metrics_df = jhu_deaths_raw_df.drop(columns = ['Province/State', 'Lat', 'Long'])
countries_metrics_df['Country/Region'] = countries_metrics_df['Country/Region'].replace('US', 'United States')

all_countries_metrics_df = pd.DataFrame(columns = ['country', 'date', 'deaths'])

for country in COUNTRIES:
    transposed_df = countries_metrics_df[countries_metrics_df['Country/Region'] == country].drop(columns = ['Country/Region']).transpose()
    num_cols = len(transposed_df.columns)
    num_rows = transposed_df.shape[0]

    if num_rows == 0 or num_cols == 0:
        continue

    value_col_name = transposed_df.columns[0]

    country_series = pd.Series([country] * num_rows)
    dates_series = pd.to_datetime(pd.Series(transposed_df.index))
    value_series = transposed_df[value_col_name].reset_index()[value_col_name]
    
    country_df = pd.DataFrame({
        'country': country_series,
        'date': dates_series,
        'deaths': value_series
    })

    all_countries_metrics_df = all_countries_metrics_df.append(country_df, ignore_index = True)

all_countries_metrics_df.to_csv(os.path.join(app_dir, 'data/staging', 'jhu_deaths.csv'), index = False)

In [39]:

jhu_cases_raw_df = pd.read_csv(os.path.join(app_dir, 'data/raw/jhu', 'time_series_covid19_confirmed_global.csv'))
countries_metrics_df = jhu_cases_raw_df.drop(columns = ['Province/State', 'Lat', 'Long'])
countries_metrics_df['Country/Region'] = countries_metrics_df['Country/Region'].replace('US', 'United States')

all_countries_metrics_cols = ['country', 'date', 'cases']
for shift in range(1, 20):
    all_countries_metrics_cols.append('cases_lag_' + str(shift))
all_countries_metrics_df = pd.DataFrame(columns = all_countries_metrics_cols)


for country in COUNTRIES:
    grouped_countries_df = countries_metrics_df.groupby('Country/Region').sum().reset_index()
    transposed_df = grouped_countries_df[grouped_countries_df['Country/Region'] == country].drop(columns = ['Country/Region']).transpose()
    num_cols = len(transposed_df.columns)
    num_rows = transposed_df.shape[0]

    if num_rows == 0 or num_cols == 0:
        continue

    value_col_name = transposed_df.columns[0]

    country_series = pd.Series([country] * num_rows)
    dates_series = pd.to_datetime(pd.Series(transposed_df.index))
    value_series = transposed_df[value_col_name].reset_index()[value_col_name]

    country_df = pd.DataFrame({
        'country': country_series,
        'date': dates_series,
        'cases': value_series
    })

    for shift in range(1, 20):
        country_df['cases_lag_' + str(shift)] = value_series.shift(shift, fill_value = 0)

    all_countries_metrics_df = all_countries_metrics_df.append(country_df, ignore_index = True)

all_countries_metrics_df.to_csv(os.path.join(app_dir, 'data/staging', 'jhu_cases.csv'), index = False)
jhu_cases_raw_df = pd.read_csv(os.path.join(app_dir, 'data/raw/jhu', 'time_series_covid19_confirmed_global.csv'))
countries_metrics_df = jhu_cases_raw_df.drop(columns = ['Province/State', 'Lat', 'Long'])
countries_metrics_df['Country/Region'] = countries_metrics_df['Country/Region'].replace('US', 'United States')

all_countries_metrics_cols = ['country', 'date', 'cases']
for shift in range(1, 20):
    all_countries_metrics_cols.append('cases_lag_' + str(shift))
all_countries_metrics_df = pd.DataFrame(columns = all_countries_metrics_cols)


for country in COUNTRIES:
    grouped_countries_df = countries_metrics_df.groupby('Country/Region').sum().reset_index()
    transposed_df = grouped_countries_df[grouped_countries_df['Country/Region'] == country].drop(columns = ['Country/Region']).transpose()
    num_cols = len(transposed_df.columns)
    num_rows = transposed_df.shape[0]

    if num_rows == 0 or num_cols == 0:
        continue

    value_col_name = transposed_df.columns[0]

    country_series = pd.Series([country] * num_rows)
    dates_series = pd.to_datetime(pd.Series(transposed_df.index))
    value_series = transposed_df[value_col_name].reset_index()[value_col_name]

    country_df = pd.DataFrame({
        'country': country_series,
        'date': dates_series,
        'cases': value_series
    })

    for shift in range(1, 20):
        country_df['cases_lag_' + str(shift)] = value_series.shift(shift, fill_value = 0)

    all_countries_metrics_df = all_countries_metrics_df.append(country_df, ignore_index = True)

all_countries_metrics_df.to_csv(os.path.join(app_dir, 'data/staging', 'jhu_cases.csv'), index = False)

In [49]:
google_mobility_df = pd.read_csv(os.path.join(app_dir, 'data/staging', 'google_mobility.csv'))
oxford_stringency_df = pd.read_csv(os.path.join(app_dir, 'data/staging', 'oxford_stringency.csv'))
jhu_cases_df = pd.read_csv(os.path.join(app_dir, 'data/staging', 'jhu_cases.csv'))
jhu_deaths_df = pd.read_csv(os.path.join(app_dir, 'data/staging', 'jhu_deaths.csv'))
jhu_recovered_df = pd.read_csv(os.path.join(app_dir, 'data/staging', 'jhu_recovered.csv'))
demographics_df = pd.read_csv(os.path.join(app_dir, 'data/staging', 'demographics_2019.csv'))


In [50]:
unified_df = google_mobility_df.merge(
    oxford_stringency_df,
    how = 'inner',
    left_on = ['country_region', 'date'],
    right_on = ['CountryName', 'Date']
).merge(
    jhu_cases_df,
    how = 'inner',
    left_on = ['country_region', 'date'],
    right_on = ['country', 'date']
).merge(
    jhu_deaths_df,
    how = 'inner',
    left_on = ['country_region', 'date'],
    right_on = ['country', 'date']
).merge(
    jhu_recovered_df,
    how = 'inner',
    left_on = ['country_region', 'date'],
    right_on = ['country', 'date']
)[[
    'country_region',
    'date',
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline',
    'C1_School closing',
    'C2_Workplace closing',
    'C3_Cancel public events',
    'C4_Restrictions on gatherings',
    'C5_Close public transport',
    'C6_Stay at home requirements',
    'C7_Restrictions on internal movement',
    'C8_International travel controls',
    'E1_Income support',
    'E2_Debt/contract relief',
    'H1_Public information campaigns',
    'H2_Testing policy',
    'H3_Contact tracing',
    'StringencyIndex',
    'cases',
    'cases_lag_1',
    'cases_lag_2',
    'cases_lag_3',
    'cases_lag_4',
    'cases_lag_5',
    'cases_lag_6',
    'cases_lag_7',
    'cases_lag_8',
    'cases_lag_9',
    'cases_lag_10',
    'cases_lag_11',
    'cases_lag_12',
    'cases_lag_13',
    'cases_lag_14',
    'cases_lag_15',
    'cases_lag_16',
    'cases_lag_17',
    'cases_lag_18',
    'cases_lag_19',
    'deaths',
    'recovered'
]]




In [51]:
unified_df.to_csv(os.path.join(app_dir, 'data/staging', 'unified.csv'), index = False)

In [84]:
demographics_df = pd.read_csv(os.path.join(app_dir, 'data/raw/demographics', 'WPP2019_TotalPopulationBySex.csv'))
demographics_2019_df = demographics_df[demographics_df['Time'] == 2019][['Location', 'PopTotal', 'PopDensity']]

In [87]:
demographics_2019_df.groupby('Location').mean().reset_index()

,Location,PopTotal,PopDensity
0,Afghanistan,38041.757,58.269
1,Africa,1308064.176,44.119
2,African Group,1306320.572,44.464
3,African Union,1306903.030,44.085
4,African Union: Central Africa,154013.705,29.192
...,...,...,...
469,World,7713468.205,59.291
470,World Bank Regional Groups (developing only),6452517.055,70.258
471,Yemen,29161.922,55.234
472,Zambia,17861.034,24.026


In [88]:
demographics_2019_df.to_csv(os.path.join(app_dir, 'data/staging', 'demographics_2019.csv'), index = False)